In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns


# **1. Read the Data**

In [ ]:
Train_data = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/train.csv')
Test_data = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/test.csv')
train_data = Train_data.copy()
test_data = Test_data.copy()

In [ ]:
train_data.head()

In [ ]:
train_date_index = train_data.set_index(pd.to_datetime(train_data['date']))

In [ ]:
# Country = Finland, Store = KaggleMart, Product = 'Kaggle Mug'
plt.figure(figsize = (30,10))
train_Finland_KaggleMart_KaggleMug = train_date_index[(train_date_index['country'] == 'Finland') & (train_date_index['store'] == 'KaggleMart') & (train_date_index['product'] == 'Kaggle Mug')]
sns.set()
sns.lineplot(x = train_Finland_KaggleMart_KaggleMug.index, y = train_Finland_KaggleMart_KaggleMug['num_sold'].values)

# **2. Data Proprecessing**

* I Split the 'date' column to 'Month' column and 'Day' column

* I define the 'weekday' and 'weekend' columns from 'date' columns.

* I dummies the 'country', 'store', 'product' columns.

In [ ]:
def minmax_Year(series):
    year = []
    for i in series:
        year.append((i - min(series)) / (max(series) - min(series)))
    return year
a = [2015, 2016, 2017, 2018, 2019]
b = minmax_Year(a)
#Year
def split_Year(date):
  return int(date.split('-')[0])

#Month
def split_month(date):
  return int(date.split('-')[1])

#Day
def split_day(date):
  return int(date.split('-')[2])

#Weekend
def weekend(date):
  import datetime
  weekend = []
  a = pd.to_datetime(date)
  for i in range(len(a)):
    if a.iloc[i].weekday() >= 5 :
      weekend.append(1)
    else:
      weekend.append(0)
  return weekend
#Weekday
def weekday(date):
    import datetime
    weekday = []
    a = pd.to_datetime(date)
    for i in range(len(a)):
        weekday.append(a.iloc[i].weekday())
    return weekday

In [ ]:
train_data['Year'] = train_data['date'].apply(split_Year)
train_data['Month'] = train_data['date'].apply(split_month)
train_data['Day'] = train_data['date'].apply(split_day)
train_data['Weekend'] = weekend(train_data['date'])
train_data['Weekday'] = weekday(train_data['date'])
trainyear_minmax = []
for i in train_data['Year']:
    trainyear_minmax.append(b[a.index(i)])
train_data['Year'] = trainyear_minmax
train_data = train_data.drop(columns = ['row_id', 'date', 'Year'])


test_data['Year'] = test_data['date'].apply(split_Year)
test_data['Month'] = test_data['date'].apply(split_month)
test_data['Day'] = test_data['date'].apply(split_day)
test_data['Weekend'] = weekend(test_data['date'])
test_data['Weekday'] = weekday(test_data['date'])
testyear_minmax = []
for i in test_data['Year']:
    testyear_minmax.append(b[a.index(i)])
test_data['Year'] = testyear_minmax
test_data = test_data.drop(columns = ['row_id', 'date','Year'])

In [ ]:
#Encoder the 'country', 'store', 'product'
#Dummies the 'country', 'store', 'product'
train_data_dum = pd.get_dummies(train_data[['country', 'store', 'product']])
test_data_dum = pd.get_dummies(test_data[['country', 'store', 'product']])

train_data = pd.concat([train_data, train_data_dum],axis = 1)
test_data = pd.concat([test_data, test_data_dum],axis = 1)

train_data = train_data.drop(columns = ['country', 'store', 'product'])
test_data = test_data.drop(columns = ['country', 'store', 'product'])

In [ ]:
train_data.info()

In [ ]:
test_data.head()

### Define the training data and training target

In [ ]:
data = train_data.drop(columns = 'num_sold')
target = train_data['num_sold']

from sklearn.preprocessing import MinMaxScaler
Normalize = MinMaxScaler()
target = np.log(target)
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, target, train_size = 0.8, random_state = 5)
x_train = Normalize.fit_transform(x_train)
x_test = Normalize.transform(x_test)


# **3. Seven Model Performance**


## 3-1. DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
sns.set()
DTR = DecisionTreeRegressor(max_depth = 60, min_samples_leaf = 2).fit(x_train, y_train)
y_pred_DTR = DTR.predict(x_test)
plt.scatter(y_test, y_pred_DTR)
plt.plot([x for x in range(4, 10)], [x for x in range(4, 10)], color = 'r')
plt.xlabel("Reality")
plt.ylabel("Predicted")
plt.title('DecisionTreeRegressor')
plt.show()
plt.clf()

## 3-2. RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
sns.set()
RFR = RandomForestRegressor(max_depth = 20, random_state = 5).fit(x_train, y_train)
y_pred_RFR = RFR.predict(x_test)
plt.scatter(y_test, y_pred_RFR)
plt.plot([x for x in range(4, 10)], [x for x in range(4, 10)], color = 'r')
plt.xlabel("Reality")
plt.ylabel("Predicted")
plt.title('RandomFroestRegressor')
plt.show()
plt.clf()

## 3-3. GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
sns.set()
GBR = GradientBoostingRegressor(learning_rate=0.10, max_depth= 5,
                                min_samples_leaf = 5,n_estimators = 500, random_state = 40,subsample = 0.3).fit(x_train, y_train)
y_pred_GBR = GBR.predict(x_test)
plt.scatter(y_test, y_pred_GBR)
plt.plot([x for x in range(4, 10)], [x for x in range(4, 10)], color = 'r')
plt.xlabel("Reality")
plt.ylabel("Predicted")
plt.title('GradientBoostingRegressor')
plt.show()
plt.clf()
print(GBR.score(x_test, y_test))

## 3-4. SVR rbf

In [ ]:
from sklearn.svm import SVR
sns.set()
svr_rbf = SVR(kernel = 'rbf').fit(x_train, y_train)
y_pred_svr = svr_rbf.predict(x_test)
plt.scatter(y_test, y_pred_svr)
plt.plot([x for x in range(4, 10)], [x for x in range(4, 10)], color = 'r')
plt.xlabel("Reality")
plt.ylabel("Predicted")
plt.title('SVM_RBF')
plt.show()
plt.clf()

## 3-5. SVR Linear

In [ ]:
from sklearn.svm import SVR
sns.set()
svr_linear = SVR(kernel = 'linear').fit(x_train, y_train)
y_pred_svr = svr_linear.predict(x_test)
plt.scatter(y_test, y_pred_svr)
plt.plot([x for x in range(4, 10)], [x for x in range(4, 10)], color = 'r')
plt.xlabel("Reality")
plt.ylabel("Predicted")
plt.title('SVM_Linear')
plt.show()
plt.clf()

## 3-6. Deep Learning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
model = Sequential()
model.add(Dense(512, input_shape = (x_train.shape[1], ), activation = 'sigmoid'))
model.add(Dense(64,activation = 'sigmoid'))
model.add(Dense(8))
model.add(Dense(1))
model.compile(loss = 'mse', optimizer = 'adam', metrics= 'mse')
history = model.fit(x_train, y_train, batch_size = 128, epochs = 100 , validation_split= 0.2, verbose = 0)

In [ ]:
import seaborn as sns
sns.set()
df_history = pd.DataFrame(history.history)
sns.lineplot(x = df_history.index, y = df_history.loss)

In [ ]:
y_pred = model.predict(x_test)
plt.scatter(y_test, y_pred)
plt.plot([x for x in range(4, 10)], [x for x in range(4, 10)], color = 'r')
plt.xlabel("Reality")
plt.ylabel("Predicted")
plt.show()
plt.clf()

## 3-7. KNeighborsRegressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
sns.set()
KNN = KNeighborsRegressor(n_neighbors = 2, weights = 'distance').fit(x_train, y_train)
y_pred_KNN = KNN.predict(x_test)
plt.scatter(y_test, y_pred_KNN)
plt.plot([x for x in range(4, 10)], [x for x in range(4, 10)], color = 'r')
plt.xlabel("Reality")
plt.ylabel("Predicted")
plt.title('KNeighborsRegressor')
plt.show()
plt.clf()

## 3-8. Xgboost Classifier

In [ ]:
import xgboost as xgb
XG = xgb.XGBRegressor(objective ='reg:linear', learning_rate = 0.1, alpha = 6, n_estimators = 500).fit(x_train, y_train)
y_pred_XG = XG.predict(x_test)
plt.scatter(y_test, y_pred_XG)
plt.plot([x for x in range(4, 10)], [x for x in range(4, 10)], color = 'r')
plt.xlabel("Reality")
plt.ylabel("Predicted")
plt.title('Xgboost Classifier')
plt.show()
plt.clf()
print(XG.score(x_test, y_test))

# **4. Eight Model Comparsion(R2_score and Mean squared error)**

In [ ]:
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
def model_fit(x_train, x_test, y_train, y_test):

  DTR = DecisionTreeRegressor(max_depth = 10, 
                              min_samples_leaf = 8).fit(x_train, y_train)

  RFR = RandomForestRegressor(max_depth = 30).fit(x_train, y_train)
    
  GBR = GradientBoostingRegressor(learning_rate=0.10, max_depth= 6,
                                min_samples_leaf = 5,n_estimators = 500, random_state = 40,subsample = 0.3).fit(x_train, y_train)

  svr_rbf = SVR(kernel = 'rbf').fit(x_train, y_train)  

  svr_linear = SVR(kernel = 'linear').fit(x_train, y_train)

  KNN = KNeighborsRegressor(n_neighbors = 10).fit(x_train, y_train)
    
  XG = xgb.XGBRegressor(objective ='reg:linear', learning_rate = 0.1, alpha = 6, n_estimators = 500).fit(x_train, y_train)

  return DTR, RFR, GBR, svr_rbf, svr_linear, KNN, XG

## 4-1. R square score Comparsion

In [ ]:
Model = model_fit(x_train, x_test, y_train, y_test)
ML_model = ['DecisionTreeRegressor', 'RandomForestRegressor', 'GradientBoostingRegressor', 'svr_rbf', 'svr_linear', 'KNeighborsRegressor','Xgboost Classifier', 'DeepLearning']
sns.set()
from sklearn.metrics import r2_score
R_square_num = []
for i in range(7):
  R_square = r2_score(y_test, Model[i].predict(x_test))
  R_square_num.append(R_square)
R_square_num.append(r2_score(y_test, model.predict(x_test)))
plt.figure(figsize = (10, 10))
plt.xlabel('R Square Score')
plt.ylabel('Model Type')
plt.title('The R Square Score Comparsion')
sns.barplot(x = R_square_num, y = ML_model)

## 4-2. Mean Square Root Comparsion

In [ ]:
Model = model_fit(x_train, x_test, y_train, y_test)
ML_model = ['DecisionTreeRegressor', 'RandomForestRegressor', 'GradientBoostingRegressor', 'svr_rbf', 'svr_linear', 'KNeighborsRegressor', 'Xgboost Classifier', 'DeepLearning']
sns.set()
from sklearn.metrics import mean_squared_error
mse_num = []
for i in range(7):
  mse = mean_squared_error(y_test, Model[i].predict(x_test))
  mse_num.append(mse)
mse_num.append(mean_squared_error(y_test, model.predict(x_test)))
plt.figure(figsize = (10, 10))
plt.xlabel('mean_square_error')
plt.ylabel('Model Type')
plt.title('The mean_square_error Comparsion')
sns.barplot(x = mse_num, y = ML_model)

# **5. Predict the Test_data**

In [ ]:
test_data = Normalize.transform(test_data)
submission_target = np.exp(Model[6].predict(test_data))

# **6. Submission**

In [ ]:
submission = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/sample_submission.csv')
submission['num_sold'] = submission_target
submission.to_csv('submission.csv', index=False)

In [ ]:
submission

# **7. Thanks for your view!**